## Read data

demonstration data set from the UEA collection

In [2]:
import numpy as np

data = [np.load(f'../data/HandMovementDirection/{variable}_{set_name}.npy')
        for variable in ['X', 'y'] for set_name in ['train', 'test']]

X_train, X_test, y_train, y_test = data

print("X_train dims: ", X_train.shape)
print("X_test dims: ", X_test.shape)

X_train dims:  (160, 10, 400)
X_test dims:  (74, 10, 400)


your X_train and X_test should be of the shape `(n_samples, n_channels, seq_len=512)` to apply the foundation model

if original sequence length is different, resize it, for example, using the following function:

In [3]:
import torch
import torch.nn.functional as F

def resize(X):
    X_scaled = F.interpolate(torch.tensor(X, dtype=torch.float), size=512, mode='linear', align_corners=False)
    return X_scaled.numpy()
    
X_train, X_test = resize(X_train), resize(X_test)

print("X_train dims: ", X_train.shape)
print("X_test dims: ", X_test.shape)

X_train dims:  (160, 10, 512)
X_test dims:  (74, 10, 512)


## Load model

In [4]:
from mantis.architecture import Mantis8M
    
device = 'cpu' # set device
network = Mantis8M(device=device) # init model
network = network.from_pretrained("paris-noah/Mantis-8M") # load weights

## Extract deep features without an adapter

By default, `.transform()` extract features for each channel independently and concatenate all the outputs in a flattened manner.

If it is needed to keep the structure, pass `three_dim=True` to `.transform()` as an argument.

In [5]:
from mantis.trainer import MantisTrainer

model = MantisTrainer(device=device, network=network) # init trainer
Z_train = model.transform(X_train)
Z_test = model.transform(X_test)

print("Z_train dims: ", Z_train.shape)
print("Z_test dims: ", Z_test.shape)

Z_train dims:  (160, 2560)
Z_test dims:  (74, 2560)


Learn a classifier.

In [6]:
from sklearn.ensemble import RandomForestClassifier

predictor = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=0)
predictor.fit(Z_train, y_train)

RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=0)

Evaluate performance.

In [7]:
y_pred = predictor.predict(Z_test)
print(f'Accuracy on the test set is {np.mean(y_test == y_pred)}')

Accuracy on the test set is 0.25675675675675674


## Extract deep features with an adapter

If the number of channels is too large, one can try to reduce dimension first and then send the transformed input to the foundation model.

Here, we will use PCA, please refer to `multichannel_adapters.ipynb` for more examples.

Apply adapter.

In [8]:
from mantis.adapters import MultichannelProjector

adapter = MultichannelProjector(new_num_channels=5, patch_window_size=1, base_projector='pca')
adapter.fit(X_train)
X_reduced_train, X_reduced_test = adapter.transform(X_train), adapter.transform(X_test)

print("X_reduced_train dims: ", X_reduced_train.shape)
print("X_reduced_test dims: ", X_reduced_test.shape)

X_reduced_train dims:  (160, 5, 512)
X_reduced_test dims:  (74, 5, 512)


Extract deep features.

In [9]:
from mantis.trainer import MantisTrainer

model = MantisTrainer(device=device, network=network) # init trainer
Z_train = model.transform(X_reduced_train)
Z_test = model.transform(X_reduced_test)

print("Z_train dims: ", Z_train.shape)
print("Z_test dims: ", Z_test.shape)

Z_train dims:  (160, 1280)
Z_test dims:  (74, 1280)


Learn a classifier.

In [10]:
from sklearn.ensemble import RandomForestClassifier

predictor = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=0)
predictor.fit(Z_train, y_train)

RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=0)

Evaluate performance.

In [11]:
y_pred = predictor.predict(Z_test)
print(f'Accuracy on the test set is {np.mean(y_test == y_pred)}')

Accuracy on the test set is 0.21621621621621623
